# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,44.498622,0.929615,1234,45.428237,43.569007,90.856474,87.138013
6,708.953277,0.150195,1234,709.103472,708.803082,1418.206943,1417.606163
7,244.598935,-0.920722,1234,243.678214,245.519657,487.356427,491.039315
8,620.783256,-0.759862,1234,620.023394,621.543119,1240.046788,1243.086237
9,16.754975,-0.939960,1234,15.815014,17.694935,31.630029,35.389869
10,10.494851,0.666237,1234,11.161088,9.828614,22.322175,19.657228
11,15.969413,0.832950,1234,16.802363,15.136463,33.604726,30.272926
12,86.493003,0.542986,1234,87.035989,85.950017,174.071977,171.900035
13,6.251204,0.398886,1234,6.650090,5.852317,13.300180,11.704634
14,543.759983,-0.056918,1234,543.703065,543.816901,1087.406129,1087.633801


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-45.428237,43.569007
6,-709.103472,708.803082
7,-243.678214,245.519657
8,-620.023394,621.543119
9,-15.815014,17.694935
10,-11.161088,9.828614
11,-16.802363,15.136463
12,-87.035989,85.950017
13,-6.650090,5.852317
14,-543.703065,543.816901


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-45.428237,43.569007
6,-709.103472,708.803082
7,-243.678214,245.519657
8,-620.023394,621.543119
9,-15.815014,17.694935
10,-11.161088,9.828614
11,-16.802363,15.136463
12,-87.035989,85.950017
13,-6.650090,5.852317
14,-543.703065,543.816901


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -45.42823688,   43.5690067 ],
       [-709.10347159,  708.80308158],
       [-243.67821362,  245.51965731],
       [-620.02339391,  621.54311865],
       [ -15.81501438,   17.69493473],
       [ -11.16108768,    9.82861425],
       [ -16.80236289,   15.13646315],
       [ -87.03598854,   85.95001726],
       [  -6.65008993,    5.8523171 ],
       [-543.70306462,  543.81690057]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,88.856474,-87.138013
6,1416.206943,-1417.606163
7,485.356427,-491.039315
8,1238.046788,-1243.086237
9,29.630029,-35.389869
10,20.322175,-19.657228
11,31.604726,-30.272926
12,172.071977,-171.900035
13,11.300180,-11.704634
14,1085.406129,-1087.633801


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
